# Mining association rules

## Dataset

We‘ll use "201707-citibike-tripdata.csv.zip" (after preprocessed in HW0)

## Schema

- Every station’s information
    - id, name, lat, lng
- Every stations’ flow data
    - id, time, in-flow, out-flow

### Import packages

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import os
import time
from plotly.graph_objs import *
from sklearn.metrics.pairwise import euclidean_distances
%matplotlib inline

### Read csv to dataframe
use pandas to read data

In [124]:
# preprocessed dataset
df = pd.read_csv('./201707-citibike-tripdata-preprocessed.csv')
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2


In [125]:
# every station's information
station_info = pd.read_csv('./station_info.csv')
station_info.head()

,station id,station name,station latitude,station logitude
0,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241
1,293,Lafayette St & E 8 St,40.730207,-73.991026
2,3242,Schermerhorn St & Court St,40.691029,-73.991834
3,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198
4,361,Allen St & Hester St,40.716059,-73.991908


In [126]:
# every station's in-flow data
station_in_flow = pd.read_csv('./in_flow.csv')
station_in_flow.head()

,72,79,82,83,116,119,120,127,128,143,...,2003,2005,2006,2008,2009,2010,2012,2021,2022,2023
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,1.0,0.0,0.0,2.0,0.0,1.0,0.0,2.0,0.0,1.0
2,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# every station's out-flow data
station_out_flow = pd.read_csv('./out_flow.csv')
station_out_flow.head()

,72,79,82,83,116,119,120,127,128,143,...,2003,2005,2006,2008,2009,2010,2012,2021,2022,2023
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Algorithm

[Apriori](https://github.com/asaini/Apriori)

It proceeds by identifying the frequent individual items in the database and extending them to larger and larger item sets as long as those item sets appear sufficiently often in the database. The frequent item sets determined by Apriori can be used to determine association rules which highlight general trends in the database: this has applications in domains such as market basket analysis.

The code attempts to implement the following paper

> Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994.

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/8eed75c18217fe2f9b15f266c40b369ce038164d)

[FP-Growth](https://github.com/enaeseth/python-fp-growth)

This module provides a pure Python implementation of the FP-growth algorithm for finding frequent itemsets. FP-growth exploits an (often-valid) assumption that many transactions will have items in common to build a prefix tree. If the assumption holds true, this tree produces a compact representation of the actual transactions and is used to generate itemsets much faster than Apriori can.

![](http://doi.ieeecomputersociety.org/cms/Computer.org/dl/trans/td/2011/09/figures/ttd2011091497x4.gif)


### Algorithm api

There are floating point inaccuracy in support and confidence, so the outcone of Apriori and FP Growth could be different

Apriori and FP Growth both calculate frequency item sets and the rules are the same, so we output rules only in Apriori

In [128]:
def Apriori(filename, support, confidence):
    s = time.time()
    print os.popen('python {}/Apriori/apriori.py -f {} -s {} -c {}'.format(os.getcwd(), filename, support, confidence)).read()
    t = time.time()
    print 'total run {} sec'.format(t - s)
    
def FP_Growth(filename, support):
    support = int(len(pd.read_csv(filename).index) * (support))
    s = time.time()
    print os.popen('python -m {}/python-fp-growth/fp_growth -s {} {}'.format(os.getcwd(), support, filename)).read()
    t = time.time()
    print 'total run {} sec'.format(t - s)

## Transaction

### First transaction

- in-flow and out-flow for station_id = 519

Distanguish in_flow and out_flow by negtive station_out_flow because (1, 0) and (0, 1) would be the same when apply algorithm

#### Use "divided 10" as discretization method

In [129]:
tx1 = pd.DataFrame([station_in_flow['519'] / 10, -station_out_flow['519'] / 10]).astype(int)

tx1 = tx1.T
tx1.columns = ['in flow', 'out flow']
tx1.to_csv('tx1.csv', index = False)
tx1.head()

,in flow,out flow
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [130]:
tx1.describe()

,in flow,out flow
count,1488.000000,1488.000000
mean,0.642473,-0.674731
std,1.307419,1.419570
min,0.000000,-11.000000
25%,0.000000,-1.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,9.000000,0.000000


#### Test the support and confidence

I think the confidence should be higher 0.5 because it will show in half frequent item sets. It makes sense!

- support 0.1 is too high, so there is only one item in each frequency item sets

In [131]:
Apriori('tx1.csv', 0.1, 0.5)
print "----------"
FP_Growth('tx1.csv', 0.1)

item: ('1',) , 0.156
item: ('-1',) , 0.156
item: ('0',) , 0.770

------------------------ RULES:

total run 0.0429608821869 sec
----------
['-1'] 233
['0'] 2079
['1'] 232

total run 0.0408051013947 sec


- support 0.01 looks like more rules

And the rules tell me the station 519 is high transportation because the in and out flow are high and balance

In [132]:
Apriori('tx1.csv', 0.01, 0.5)
print "----------"
FP_Growth('tx1.csv', 0.01)

item: ('3', '-2') , 0.010
item: ('6',) , 0.011
item: ('2', '-1') , 0.011
item: ('-6',) , 0.011
item: ('-4', '4') , 0.011
item: ('1', '-2') , 0.013
item: ('5',) , 0.014
item: ('4',) , 0.018
item: ('-4',) , 0.021
item: ('3', '-3') , 0.023
item: ('2', '-2') , 0.029
item: ('-3',) , 0.038
item: ('3',) , 0.041
item: ('-2',) , 0.054
item: ('2',) , 0.054
item: ('1', '0') , 0.068
item: ('0', '-1') , 0.071
item: ('1', '-1') , 0.074
item: ('1',) , 0.156
item: ('-1',) , 0.156
item: ('0',) , 0.770

------------------------ RULES:
Rule: ('-4',) ==> ('4',) , 0.531
Rule: ('2',) ==> ('-2',) , 0.537
Rule: ('-2',) ==> ('2',) , 0.537
Rule: ('3',) ==> ('-3',) , 0.557
Rule: ('-3',) ==> ('3',) , 0.607
Rule: ('4',) ==> ('-4',) , 0.630

total run 0.0478940010071 sec
----------
['-1'] 233
['-1', '1'] 110
['-1', '2'] 16
['-2'] 80
['-2', '3'] 15
['-3'] 56
['-4'] 32
['-4', '4'] 17
['-6'] 17
['0'] 2079
['0', '-1'] 106
['0', '1'] 101
['1'] 232
['1', '-2'] 20
['2'] 80
['2', '-2'] 43
['2', '-3'] 14
['3'] 61
['3', '-3'

#### Use "divided 20" as discretization method

In [133]:
tx2 = pd.DataFrame([station_in_flow['519'] / 20, -station_out_flow['519'] / 20]).astype(int)

tx2 = tx2.T
tx2.columns = ['in flow', 'out flow']
tx2.to_csv('tx2.csv', index = False)
tx2.head()

,in flow,out flow
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [134]:
tx2.describe()

,in flow,out flow
count,1488.000000,1488.000000
mean,0.212366,-0.230511
std,0.583852,0.647760
min,0.000000,-5.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,4.000000,0.000000


#### Test the support and confidence

I think the confidence should be higher 0.5 because it will show in half frequent item sets. It makes sense!

- test support 0.02 and the rules are almost the same as the "divided 10" discretization method

In [135]:
Apriori('tx2.csv', 0.02, 0.5)
print "----------"
FP_Growth('tx2.csv', 0.02)

item: ('-3',) , 0.020
item: ('2', '-2') , 0.021
item: ('-2',) , 0.030
item: ('2',) , 0.032
item: ('1', '-1') , 0.071
item: ('-1',) , 0.091
item: ('1',) , 0.095
item: ('0',) , 0.869

------------------------ RULES:
Rule: ('2',) ==> ('-2',) , 0.646
Rule: ('-2',) ==> ('2',) , 0.705
Rule: ('1',) ==> ('-1',) , 0.752
Rule: ('-1',) ==> ('1',) , 0.779

total run 0.0436761379242 sec
----------
['-1'] 136
['-2'] 44
['-3'] 30
['0'] 2544
['1'] 141
['1', '-1'] 106
['2'] 48
['2', '-2'] 31

total run 0.0660769939423 sec


#### Top 3 rules

- ('2',) ==> ('-2',)
- ('3',) ==> ('-3',)
- ('4',) ==> ('-4',)

sort by the confidence

#### Observation

We can join more features so that the mining rules will be more meaningful and the about the algorithm runtime are a little bit weird. I think it's implement could be complex, so the overhead take too long time when dataset is not big enough